In [210]:
#CNN is a form of deep learning that specializes in pattern recognition 

#where are the convolutional, pooling, and flattening layers in the code 

askpython.com:

The MNIST dataset is a large database of handwritten digits. 
It commonly used for training various image processing systems. 

MNIST is short for Modified National Institute of Standards and Technology database.

This dataset is used for training models to recognize handwritten digits. This has an application in scanning for 
handwritten pin-codes on letters.

MNIST contains a collection of 70,000, 28 x 28 images of handwritten digits from 0 to 9.

MNIST is popular for a multitude of reasons, these are :

MNSIT dataset is publicly available.
The data requires little to no processing before using.
It is a voluminous dataset.
Additionally, this dataset is commonly used in courses on image processing and machine learning.

(easiest way is to load through Keras)

Now that you have imported the MNIST dataset, you can use it for image classification.

When it comes to the task of image classification, nothing can beat Convolutional Neural Networks (CNN). 
CNN contains Convolutional Layers, Pooling Layers, and Flattening Layers.

Let’s see what each of these layers do.

1. Convolution Layer

Convolution layer filters the image with a smaller pixel filter. This decreases the size of the image without 
losing the relationship between pixels.

2. Pooling Layer

The main job of the pooling layer is to reduce the spatial size of the image after convolution.

A pooling layer reduces the amount of parameters by selecting the maximum, average, or sum values inside the pixels.

Max pooling is the most commonly used pooling technique.

3. Flattening Layer

A flattening layer represents the multi-dimensional pixel vector as a one-dimensional pixel vector.

Slides (info from ppl in kaggle): 

The purpose of this exercise is to see how accurate of a Neural Network we can create to classify X-Ray scans 
from patients with pneumonia.

This notebook tackles pneumonia classification using CNN (Convolutional Neural Network). 
In addition, this will also experiment with threshold values. 

Pneumonia is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli. 
Typically, symptoms include some combination of productive or dry cough, chest pain, fever and difficulty 
breathing.

(2nd tab has good graphs and visuals...3rd tab has good explanations, 4th one had good explanations and visuals too)

What I will do: 
-Purpose slide
-Explanation slide of Pneumonia and pictures
-Explanation slide of CNN? 
-conf matrix and graphs before and after tuning 

In [211]:
pwd

'/Users/daniellejenkins17/Documents/GitHub/Pneumonia-Classification'

In [212]:
import numpy as np             
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

In [213]:
import glob
import random
from random import randint
from pathlib import Path

In [214]:
import numpy as np # forlinear algebra
import matplotlib.pyplot as plt #for plotting things
import os
from PIL import Image

In [215]:
# Keras Libraries
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix

In [216]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array

from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer

In [217]:
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

In [244]:
train_data_dir = "data//train"
test_data_dir = "data//test"
val_data_dir = "data//val"

In [219]:
trn_data = image_dataset_from_directory(train_data_dir,batch_size = 32)
tst_data = image_dataset_from_directory(test_data_dir)
val_data = image_dataset_from_directory(val_data_dir)

#the classes are pneumonia or no pneumonia 

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [220]:
for data, label in trn_data.take(1):
    print(label.shape)
    print(label)


(32,)
tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0], shape=(32,), dtype=int32)


In [230]:
trn_data

<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>

In [245]:
#Eric's: #5216 is batch size for training set

# get all the data in the directory split/train, and reshape them
# normalizees by rescaling
# set image size to 256 x 256
# batch size varies for each set size

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                     train_data_dir, 
                                                     target_size=(224, 224),
                                                     classes = ['NORMAL', 'PNEUMONIA'],
                                                     batch_size=5216) # 5216 total from train

# get all the data in the directory split/validation, and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                   val_data_dir, 
                                                   target_size=(224, 224),
                                                   classes = ['NORMAL', 'PNEUMONIA'],
                                                   batch_size = 16) # 16 total from val

# get all the data in the directory split/test, and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                    test_data_dir, 
                                                    target_size=(224, 224),
                                                    classes = ['NORMAL', 'PNEUMONIA'],
                                                    batch_size = 624) # 624 total from test

Found 5217 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [246]:
#do labeling 

train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [251]:
train_images.shape

(5216, 224, 224, 3)

In [252]:
test_images.shape

(624, 224, 224, 3)

In [253]:
#reshape data so it'll go in the neural network. It only accepts it as a certain size...?


train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)
print("Train", train_img.shape)
print("Validation", val_img.shape)
print("Test", test_img.shape)

train_y = np.reshape(train_labels[:,1], (5216,1))
test_y = np.reshape(test_labels[:,1], (624,1))
val_y = np.reshape(val_labels[:,1], (16,1))

Train (5216, 150528)
Validation (16, 150528)
Test (624, 150528)


In [256]:
train_labels

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [257]:
train_y = np.reshape(train_labels[:,1], (5216, 1))
test_y = np.reshape(test_labels[:,1], (624, 1))
val_y = np.reshape(val_labels[:,1], (16, 1))

In [258]:
test_y

array([[1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],

In [262]:
model= Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer="adam")

In [263]:
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [266]:
baseline_cnn = model.fit(train_images, train_y, epochs= 20, validation_data=(val_images, val_y), batch_size=64)

Epoch 1/20
82/82 [==============================] - 184s 2s/step - loss: 0.2812 - accuracy: 0.8811 - val_loss: 0.4893 - val_accuracy: 0.7500
Epoch 2/20
82/82 [==============================] - 245s 3s/step - loss: 0.1126 - accuracy: 0.9601 - val_loss: 0.3632 - val_accuracy: 0.7500
Epoch 3/20
82/82 [==============================] - 251s 3s/step - loss: 0.0822 - accuracy: 0.9697 - val_loss: 0.6688 - val_accuracy: 0.6875
Epoch 4/20
82/82 [==============================] - 166s 2s/step - loss: 0.0820 - accuracy: 0.9695 - val_loss: 0.3599 - val_accuracy: 0.8125
Epoch 5/20
82/82 [==============================] - 209s 3s/step - loss: 0.0525 - accuracy: 0.9812 - val_loss: 0.5316 - val_accuracy: 0.6875
Epoch 6/20
82/82 [==============================] - 206s 3s/step - loss: 0.0445 - accuracy: 0.9847 - val_loss: 0.4598 - val_accuracy: 0.6875
Epoch 7/20
82/82 [==============================] - 192s 2s/step - loss: 0.0437 - accuracy: 0.9856 - val_loss: 0.1971 - val_accuracy: 0.9375
Epoch 8/20
82

In [267]:
baseline_cnn.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [249]:
#Conv2D takes tensors of shape:
#(image_height, image_width, color_channels) for each image
#MNIST has one color channel (grayscale).

In [235]:
#reshape data first 

train_data_dir = train_data_dir.reshape((60000, 28, 28, 1))
test_data_dir = test_data_dir.reshape((10000, 28, 28, 1))

In [236]:
# Normalize pixel values to be between 0 and 1
train_data_dir, test_data_dir = train_data_dir / 255, test_data_dir / 255

In [227]:
#next step is normalizing the data 

#scale at 255 

#figure out what convolution vs pooling is 

In [237]:
model = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1)))
# max pool in 2x2 window
model.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# transition to dense fully-connected part of network
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [238]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)               

In [239]:
#compiling and training the model

model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

#training accuracy 

In [240]:
xrays = model.fit(trn_data, train_data_dir, epochs= 20, validation_split = 0.2, batch_size=32)

Epoch 1/20


ValueError: in user code:

    /opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/learn-env

In [147]:
#evaluating the model (test accuracy)

_, test_acc = model.evaluate(tst_data, to_categorical(test_data_dir))

313/313 [==============================] - 1s 4ms/step - loss: 0.0630 - accuracy: 0.9897


In [157]:
# predict method outputs probability of
# being class 1

#preprocessing for confusion matrix 

y_proba = model.predict(tst_data) 
y_proba[0:5]

array([[1.4271168e-29, 2.0479261e-22, 3.2090973e-21, 5.5235569e-16,
        7.8843184e-24, 9.3504293e-26, 3.4058755e-30, 1.0000000e+00,
        1.7210804e-25, 1.1735507e-23],
       [3.4451508e-25, 2.8826293e-18, 1.0000000e+00, 2.2258897e-28,
        2.3966679e-25, 9.4795409e-34, 4.1667209e-27, 1.2872423e-23,
        1.1073031e-25, 8.1995217e-31],
       [3.1269735e-12, 9.9999988e-01, 1.6031081e-12, 6.5236397e-12,
        1.4505068e-09, 9.8476136e-08, 4.9690543e-11, 2.2211408e-08,
        3.9227882e-10, 1.2633061e-11],
       [1.0000000e+00, 4.6766319e-18, 3.4828798e-16, 3.0548942e-15,
        1.7819830e-18, 4.6977040e-14, 4.1034481e-14, 9.7089153e-15,
        4.1129273e-15, 5.1908866e-14],
       [1.2113141e-26, 2.6926158e-22, 6.6787875e-24, 8.9509893e-24,
        1.0000000e+00, 5.0332272e-24, 1.8922509e-24, 5.2589982e-19,
        4.5586117e-21, 4.9928786e-16]], dtype=float32)

In [158]:
# getting class predictions
y_pred = (y_proba > 0.5).astype('int')
y_pred[0:5]

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [165]:
test_data_dir

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [162]:
#making confusion matrix work 

cm = confusion_matrix(test_data_dir, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

disp.plot(cmap=plt.cm.Blues)
plt.show()

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [ ]:
#improving the model with regularization

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

# a new layer that rescales/normalizes the activations after each layer.
from tensorflow.keras.layers import BatchNormalization

In [ ]:
model2 = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1), kernel_regularizer = l2(1e-2) ))

model2.add(BatchNormalization())
# max pool in 2x2 window
model2.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(1e-2) ))

model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer = l2(1e-2)))

model2.add(BatchNormalization())

# transition to dense fully-connected part of network
model2.add(Flatten())
model2.add(Dense(64, activation='relu', kernel_regularizer = l2(1e-2)))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
history_reg = model2.fit(trn_data, train_data_dir, epochs= 20, validation_split = 0.2, batch_size=64)